In [17]:
import pandas as pd
import numpy as np
import os
import datetime as dt
from datetime import date
from datetime import datetime
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor

In [18]:
final = pd.DataFrame(columns = ["ORDER_NO","ORDER_DATE","ORDER_TYPE","LEVEL_OF_SERVICE","ORIGINAL_TOTAL_AMOUNT","PAYMENT_TYPE","STATUS_NAME","SHIPNODE_KEY"])
for the_file in os.listdir(r"C:\Users\luis.montoya\Downloads\Resumen Mes 2022"):
    archivo = os.path.join(r"C:\Users\luis.montoya\Downloads\Resumen Mes 2022",the_file)
    parcial = pd.read_csv(archivo,sep = ",")
    columnas_inicial = ["RN","ORDER_HEADER_KEY","ENTRY_TYPE","ORDER_NO","DOCUMENT_TYPE","ORDER_DATE","ORDER_TYPE","ENTRY_TYPE2","LEVEL_OF_SERVICE","CUSTOMER_PHONE_NO","EXTN_RUN_RUT_NIT","ORIGINAL_TOTAL_AMOUNT","ENTERPRISE_KEY","PAYMENT_TYPE","STATUS_NAME","EXTN_ORG_REQ_SHIP_DATE","STATUS_DATE","EXTN_ET_FULFILMENT","SHIPNODE_KEY"]
    iteracion = len(parcial.columns) - len(columnas_inicial)
    for i in range(iteracion):
        columnas_inicial.append("BLANCO")
        
    parcial.columns = columnas_inicial
    parcial = parcial.drop(0,axis = 0)
    print(the_file)
    parcial = parcial[["ORDER_NO","ORDER_DATE","ORDER_TYPE","LEVEL_OF_SERVICE","ORIGINAL_TOTAL_AMOUNT","PAYMENT_TYPE","STATUS_NAME","SHIPNODE_KEY"]]
    final = pd.concat([final,parcial],axis = 0)
    
    final = final.drop_duplicates(subset = ["ORDER_NO","STATUS_NAME"])


Abril 2022(1).csv
Abril 2022(2).csv
Abril 2022(3).csv
Abril 2022(4).csv
Abril 2022(5).csv
Abril 2022(6).csv
Abril 2022(7).csv
Agosto 2022 (2).csv
Agosto 2022 (3).csv
Agosto 2022 (4).csv
Agosto 2022 (5).csv
Agosto 2022 (6).csv
Agosto 2022 (7).csv
Agosto 2022 (8).csv
Agosto 2022.csv
Diciembre 2022(1).csv
Diciembre 2022(10).csv
Diciembre 2022(2).csv
Diciembre 2022(3).csv
Diciembre 2022(4).csv
Diciembre 2022(5).csv
Diciembre 2022(6).csv
Diciembre 2022(7).csv
Diciembre 2022(8).csv
Diciembre 2022(9).csv
Enero 2022(1).csv
Enero 2022(2).csv
Enero 2022(3).csv
Enero 2022(4).csv
Enero 2022(5).csv
Enero 2022(6).csv
Enero 2022(7).csv
Enero 2022(8).csv
Enero 2023(1).csv
Febrero 2022(1).csv
Febrero 2022(2).csv
Febrero 2022(3).csv
Febrero 2022(4).csv
Febrero 2022(5).csv
Febrero 2022(6).csv
Febrero 2022(7).csv
Julio 2022 (1).csv
Julio 2022 (2).csv
Julio 2022 (3).csv
Julio 2022 (4).csv
Julio 2022 (5).csv
Julio 2022 (6).csv
Julio 2022 (7).csv
Julio 2022 (8).csv
Junio 2022 (1).csv
Junio 2022 (2).csv
Junio

In [19]:
final = final[final["SHIPNODE_KEY"]=="COCV_816"]
final = final[final.STATUS_NAME.isin(['Delivered To Customer','Packing Complete','In Transit To Customer','Return Received','Return Created'])]

final["ORDER_DATE"] = pd.to_datetime(final["ORDER_DATE"],format="%Y-%m-%d %H:%M:%S")

fechas = final["ORDER_DATE"]
hora_orden = []
fecha_nuevo = []
anio = []
dia_orden = []
mes_orden = []

for elemento in fechas:
    resta = elemento -dt.timedelta(hours=5)
    fecha_nuevo.append(resta)
    hora_orden.append(resta.hour)
    anio.append(resta.year)
    mes_orden.append(resta.month)
    dia_orden.append(resta.day)
    
final["Hora_Orden"] = hora_orden
final["ORDER_DATE_COL"] = fecha_nuevo
final["Anio_Orden"] = anio
final["Dia_Orden"] = dia_orden
final["Mes_Orden"] = mes_orden



final = final.sort_values(by =["ORDER_DATE_COL"],ascending = True)
final["indice_c"] = ""

for i in range(0,len(final)):
    final.iloc[i,13] = str(final.iloc[i,11]) +"/" +  str(final.iloc[i,12]) + "/" + str(final.iloc[i,10]) + " " + str(final.iloc[i,8]) + ":" + "00:00"
    
fechas_previas = final["indice_c"] 
fechas_convert = []


for elemento in fechas_previas:
    convertido = datetime.strptime(elemento, '%d/%m/%Y %H:%M:%S')
    fechas_convert.append(convertido)

final["indice_c"]  = fechas_convert   

data_1= pd.pivot_table(final,index = ["indice_c"], values = ["ORDER_NO"], aggfunc = "count")
data_1=data_1.reset_index()

In [20]:
limite_sup = date.today()
limite_sup =limite_sup - dt.timedelta(days = 30)
limite_sup = str(limite_sup.day) + "/" + str(limite_sup.month) +"/" + str(limite_sup.year) + " 23:00:00"
limite_sup = datetime.strptime(limite_sup,'%d/%m/%Y %H:%M:%S')
limite_sup_t = str(limite_sup.month)  + "/" + str(limite_sup.day) + "/"+  str(limite_sup.year) + " 23:00:00"


limite_inf_prueba = limite_sup + dt.timedelta(hours= 1)
limite_inf_prueba_t = str(limite_inf_prueba.month) + "/" +  str(limite_inf_prueba.day) + "/"  + str(limite_inf_prueba.year) + " 00:00:00"
limite_sup_prueba = date.today()
limite_sup_prueba = limite_sup_prueba -dt.timedelta(days = 7)
limite_sup_prueba = str(limite_sup_prueba.day) + "/" + str(limite_sup_prueba.month) + "/" +  str(limite_sup_prueba.year) + " 23:00:00"
limite_sup_prueba = datetime.strptime(limite_sup_prueba,'%d/%m/%Y %H:%M:%S')
limite_sup_prueba_t = str(limite_sup_prueba.month) + "/" + str(limite_sup_prueba.day)  + "/"+  str(limite_sup_prueba.year) + " 23:00:00"



limite_inf = limite_sup - dt.timedelta(days=330)
limite_inf = str(limite_inf.day) + "/" + str(limite_inf.month) + "/" + str(limite_inf.year) + " 00:00:00"
limite_inf = datetime.strptime(limite_inf,'%d/%m/%Y %H:%M:%S')
limite_inf_t =   str(limite_inf.month)  + str(limite_inf.day)  +"/" + str(limite_inf.year) + " 00:00:00"



indices = []
indices.append(limite_inf)                       
                                                        
for i in range(0,8495):
    limite_inf = limite_inf + dt.timedelta(hours = 1)
    indices.append(limite_inf)


df_indices = pd.DataFrame()
df_indices["indices"] = indices

df_indices = pd.merge(df_indices,data_1,left_on = "indices",right_on ="indice_c",how = "left")
df_indices = df_indices[["indices","ORDER_NO"]]
df_indices = df_indices.fillna(0)
df_indices.index = df_indices["indices"]

entrenamiento = df_indices.loc[:limite_sup_t]
prueba = df_indices.loc[limite_inf_prueba_t:limite_sup_prueba_t]

data_train =  entrenamiento["ORDER_NO"] 
data_test =  prueba["ORDER_NO"] 

secuencia_fig = go.Figure()
secuencia_fig.add_trace(go.Scatter(x = df_indices["indices"], y = df_indices["ORDER_NO"]))
secuencia_fig.update_layout(xaxis=dict(rangeslider = dict(visible = True),type = "date"))
secuencia_fig.show()

In [21]:
scaler = MinMaxScaler()
data_train_scaled = scaler.fit_transform(np.array(data_train).reshape(-1,1))
data_train_scaled = [u[0] for u in data_train_scaled]

data_test_scaled = scaler.fit_transform(np.array(data_test).reshape(-1,1))
data_test_scaled = [u[0] for u in data_test_scaled]


df_indices_train_scaled = pd.DataFrame()
indices_train = entrenamiento["indices"]
df_indices_train_scaled["indices"] = indices_train
df_indices_train_scaled["Cantidad_Ordenes"] = data_train_scaled
df_indices_train_scaled.index = df_indices_train_scaled["indices"]


df_indices_test_scaled = pd.DataFrame()
indices_test = prueba["indices"]
df_indices_test_scaled["indices"] = indices_test
df_indices_test_scaled["Cantidad_Ordenes"] = data_test_scaled
df_indices_test_scaled.index = df_indices_test_scaled["indices"]


vector_ent = df_indices_train_scaled["Cantidad_Ordenes"]
vector_test = df_indices_test_scaled["Cantidad_Ordenes"]

In [22]:
def sliding_time(ts, window_size=168):
    n = ts.shape[0] - window_size  
    X = np.empty((n, window_size))
    y = np.empty(n)
    
    for i in range(window_size, ts.shape[0]):   
        y[i - window_size] = ts[i]
        X[i- window_size, 0:window_size] = np.array(ts[i - window_size:i])
    
    return X, y

X, observed_scaled=sliding_time(vector_ent,168)



tsp = TimeSeriesSplit(n_splits = 5)
params = {
    'hidden_layer_sizes' : [(120,60,30),(50,20,10),(20,10,5),(10)],
    'activation': ['logistic','tanh','relu']
    
}

mlp = GridSearchCV(estimator = MLPRegressor(
    solver = 'lbfgs',
    max_iter = 10000,
    n_iter_no_change = 50,
    validation_fraction = 0.2),
                   
                   cv = tsp,
                   param_grid = params)
    

    

mlp.fit(X, observed_scaled)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=MLPRegressor(max_iter=10000, n_iter_no_change=50,
                                    solver='lbfgs', validation_fraction=0.2),
             param_grid={'activation': ['logistic', 'tanh', 'relu'],
                         'hidden_layer_sizes': [(120, 60, 30), (50, 20, 10),
                                                (20, 10, 5), 10]})

In [23]:
mlp.best_params_

{'activation': 'tanh', 'hidden_layer_sizes': (50, 20, 10)}

In [24]:
X_test, observed_scaled_test=sliding_time(vector_test,168)

y_pred_scaled = mlp.predict(X_test)
y_pred_scaled = y_pred_scaled.tolist()

y_pred_origin = scaler.inverse_transform([[u] for u in y_pred_scaled])
y_pred_origin = [u[0] for u in y_pred_origin]

observed_test_origin = scaler.inverse_transform([[u] for u in observed_scaled_test])
observed_test_origin = [u[0] for u in observed_test_origin]

In [25]:
secuencia_pred = go.Figure()
secuencia_pred.add_trace(go.Scatter(x = df_indices_test_scaled["indices"],y = [None]*168 +y_pred_origin, name="predicción"))
secuencia_pred.add_trace(go.Scatter(x = df_indices_test_scaled["indices"],y =[None]*168 + observed_test_origin, name ="observaciones"))
secuencia_pred.update_layout(xaxis=dict(rangeslider = dict(visible = True),type = "date"))
secuencia_pred.show()

In [26]:
limite_sup_pred = date.today()- dt.timedelta(days = 1)
limite_sup_pred = str(limite_sup_pred.day) + "/" + str(limite_sup_pred.month) + "/" + str(limite_sup_pred.year) + " 23:00:00"
limite_sup_pred = datetime.strptime(limite_sup_pred,'%d/%m/%Y %H:%M:%S')

f_predic = []
f_predic.append(limite_sup_pred)
f_predic_atras = []
f_predic_atras.append(limite_sup_pred)
limite_sup_pred2 = limite_sup_pred


for i in range(0,168):
    limite_sup_pred = limite_sup_pred + dt.timedelta(hours = 1)
    f_predic.append(limite_sup_pred)
    
for i in range(0,719):
    limite_sup_pred2 = limite_sup_pred2 - dt.timedelta(hours = 1)
    f_predic_atras.append(limite_sup_pred2)

df_indices_pred = pd.DataFrame()
df_indices_pred["indices"] = f_predic
df_indices_pred = df_indices_pred.sort_values(by = ["indices"], ascending =True)    

df_indices_pred_atras = pd.DataFrame()
df_indices_pred_atras["indices"] = f_predic_atras

df_indices_pred_atras = pd.merge(df_indices_pred_atras,data_1,left_on = "indices",right_on = "indice_c",how = "left")
df_indices_pred_atras = df_indices_pred_atras[["indices","ORDER_NO"]]
df_indices_pred_atras = df_indices_pred_atras.fillna(0)
df_indices_pred_atras = df_indices_pred_atras.sort_values(by  = ["indices"],ascending = True)


data_pred =  df_indices_pred_atras["ORDER_NO"]

scaler2 = MinMaxScaler()
data_scaled_pred = scaler2.fit_transform(np.array(data_pred).reshape(-1,1))
data_scaled_pred = [u[0] for u in data_scaled_pred]

df_indices_pred_atras["ORDER_NO_esc"] = data_scaled_pred

vector_prediccion = df_indices_pred_atras["ORDER_NO_esc"]


X_pred_real, observed_scaled_r=sliding_time(vector_prediccion ,168)
ultimos_reales = X_pred_real[len(X_pred_real)-1:]
y_predicciones = []


for i in range(0,169):
    ultima_pred = mlp.predict(ultimos_reales)
    y_predicciones.append(ultima_pred[0])
    ultimos_reales = np.roll(ultimos_reales,-1)
    ultimos_reales[0,-1] = ultima_pred[0]
    
    
y_pred_real_origin = scaler.inverse_transform([[u] for u in y_predicciones])
y_pred_real_origin = [u[0] for u in y_pred_real_origin]


secuencia_pred_real = go.Figure()
secuencia_pred_real.add_trace(go.Scatter(x = df_indices_pred["indices"] ,y = y_pred_real_origin, name="predicción"))
secuencia_pred_real.update_layout(xaxis=dict(rangeslider = dict(visible = True),type = "date"))
secuencia_pred_real.show()

C:\Users\luis.montoya\AppData\Local\Temp\ipykernel_15740\3954442315.py:8: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.



In [27]:
df_pred_x = pd.DataFrame(X_pred_real)
df_pred_x.to_excel(r"C:\Users\luis.montoya\Downloads\dataframe.xlsx")
df_indices_pred_atras.to_excel(r"C:\Users\luis.montoya\Downloads\data_prediccion.xlsx")